In [1]:
#加载飞桨、Numpy和相关类库
import paddle                            # 导入paddle模块
import paddle.fluid as fluid
import paddle.fluid.dygraph as dygraph
from paddle.fluid.dygraph import FC
import numpy as np
import os
import random

## 1 数据处理（Python实现）

In [2]:
def load_data():
    # 从文件导入数据
    datafile = './housing.data'
    data = np.fromfile(datafile, sep=' ')

    # 每条数据包括14项，其中前面13项是影响因素，第14项是相应的房屋价格中位数
    feature_names = [ 'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', \
                      'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV' ]
    feature_num = len(feature_names)

    # 将原始数据进行Reshape，变成[N, 14]这样的形状
    data = data.reshape([data.shape[0] // feature_num, feature_num])

    # 将原数据集拆分成训练集和测试集
    # 这里使用80%的数据做训练，20%的数据做测试
    # 测试集和训练集必须是没有交集的
    ratio = 0.8
    offset = int(data.shape[0] * ratio)
    training_data = data[:offset]

    # 计算train数据集的最大值，最小值，平均值
    maximums, minimums, avgs = training_data.max(axis=0), training_data.min(axis=0), \
                                 training_data.sum(axis=0) / training_data.shape[0]
    
    # 记录数据的归一化参数，在预测时对数据做归一化
    global max_values
    global min_values
    global avg_values
    max_values = maximums
    min_values = minimums
    avg_values = avgs

    # 对数据进行归一化处理
    for i in range(feature_num):
        #print(maximums[i], minimums[i], avgs[i])
        data[:, i] = (data[:, i] - avgs[i]) / (maximums[i] - minimums[i])

    # 训练集和测试集的划分比例
    #ratio = 0.8
    #offset = int(data.shape[0] * ratio)
    training_data = data[:offset]
    test_data = data[offset:]
    return training_data, test_data

## 2 模型设计（建议Python实现）
定义线性回归的网络结构，飞桨建议通过创建Python类的方式完成模型网络的定义。

在类的初始化函数中定义每一层网络的实现函数，这里我们定义了一层全连接层FC，模型结构和1-2 节模型保持一致。  
定义forward函数构建神经网络结构，实现前向计算过程，并返回预测结果，本例中返回的是房价预测结果。

用的是 Python 2 的话，super() 替换为 suepr(Class, self) 

In [3]:
# fluid.dygraph.Layer: 基于OOD实现的动态图Layer，包含该Layer的参数、前序运行的结构等信息。
class Regressor(fluid.dygraph.Layer):
    def __init__(self, name_scope):
        super(Regressor, self).__init__(name_scope)
        name_scope = self.full_name()
        print(name_scope)
        # 定义一层全连接层，输出维度是1，激活函数为None，即不使用激活函数
        self.fc = FC(name_scope, size=1, act=None)
    
    # 网络的前向计算函数
    def forward(self, inputs):
        x = self.fc(inputs)
        return x

## 3 训练配置
训练配置包括：

声明定义好的模型。  
加载训练数据和测试数据。  
设置优化算法和学习率，本次实验优化算法使用随机梯度下降SGD，学习率使用 0.01。   
说明：  
在之前基于Python实现神经网络模型的案例中，我们为实现梯度下降编写了大量代码，而使用飞桨框架可以大大简化这个过程。

### with fluid.dygraph.guard()创建了飞桨动态图的工作环境，在该环境下完成模型声明、数据转换、以及模型训练等操作。

In [4]:
# 定义飞桨动态图的工作环境
# place (fluid.CPUPlace|fluid.CUDAPlace, 可选) – 动态图执行的设备，可以选择cpu，gpu，
# 如果用户未制定，则根据用户paddle编译的方式来选择运行的设备，如果编译的cpu版本，则在cpu上运行，
# 如果是编译的gpu版本，则在gpu上运行。默认值：None。
with fluid.dygraph.guard():
    # 声明定义好的线性回归模型
    model = Regressor("Regressor")
    # 开启模型训练模式
    model.train()
    # 加载数据
    training_data, test_data = load_data()
    # 定义优化算法，这里使用随机梯度下降-SGD
    # 学习率设置为0.01
    opt = fluid.optimizer.SGD(learning_rate=0.05)

Regressor/Regressor_0


## 4 训练过程
完成训练配置后即可启动训练过程。训练采用二层循环嵌套方式：

**内层循环**负责整个数据集的一次遍历，遍历数据集采用分批次（batch）方式，假设数据集样本数量为1000，一个批次有10个样本，则遍历一次数据集的批次数量是1000/10=100，即内层循环需要循环100次；  
**外层循环**定义遍历数据集的次数，本次训练中外层循环10次，通过参数EPOCH_NUM设置；  

说明:  
batch大小的选择会影响训练效果，batch过大会增大内存消耗，过小则每个batch的样本数据没有统计意义。本次训练数据集较小，我们设置batch为10。

在每次内层循环都需要进行前向计算、损失函数计算和梯度反向传播三个步骤，计算的过程与Python编写的模型完全一致；

前向计算即将一个批次的样本数据灌入网络中，计算输出结果。  
以前向计算结果和真实房价作为输入，通过损失函数square_error_cost计算出损失函数值（Loss）。  
执行梯度反向传播backward函数，即从后到前逐层计算每一层的梯度，并根据设置的优化算法更新参数（opt.minimize）。  
这个实现过程令人惊喜，前向计算、计算损失和反向传播梯度，每个操作居然只有1-2行代码即可实现！我们再也不用一点点的实现模型训练的细节，这就是使用飞桨框架的威力！

In [5]:
with dygraph.guard():
    EPOCH_NUM = 50   # 设置外层循环次数
    BATCH_SIZE = 200  # 设置batch大小
    
    # 定义外层循环
    for epoch_id in range(EPOCH_NUM):
        # 在每轮迭代开始之前，将训练数据的顺序随机的打乱
        np.random.shuffle(training_data)
        # 将训练数据进行拆分，每个batch包含10条数据
        mini_batches = [training_data[k:k+BATCH_SIZE] for k in range(0, len(training_data), BATCH_SIZE)]
        # 定义内层循环
        for iter_id, mini_batch in enumerate(mini_batches):
            x = np.array(mini_batch[:, :-1]).astype('float32') # 获得当前批次训练数据
            y = np.array(mini_batch[:, -1:]).astype('float32') # 获得当前批次训练标签（真实房价）
            # 将numpy数据转为飞桨动态图variable形式
            house_features = dygraph.to_variable(x)
            prices = dygraph.to_variable(y)
            
            # 前向计算
            predicts = model(house_features)
            
            # 计算损失
            loss = fluid.layers.square_error_cost(predicts, label=prices)
            avg_loss = fluid.layers.mean(fluid.layers.sqrt(loss))
            if iter_id%20==0:
                print("epoch: {}, iter: {}, loss is: {}".format(epoch_id, iter_id, avg_loss.numpy()))
            
            # 反向传播
            avg_loss.backward()
            # 最小化loss,更新参数
            # opt 优化方法 SGD最小化
            opt.minimize(avg_loss)
            # 清除梯度
            model.clear_gradients()
    # 保存模型
    fluid.save_dygraph(model.state_dict(), 'LR_model')

epoch: 0, iter: 0, loss is: [0.39176014]
epoch: 1, iter: 0, loss is: [0.35759827]
epoch: 2, iter: 0, loss is: [0.37494797]
epoch: 3, iter: 0, loss is: [0.33708817]
epoch: 4, iter: 0, loss is: [0.3066762]
epoch: 5, iter: 0, loss is: [0.29184678]
epoch: 6, iter: 0, loss is: [0.30939215]
epoch: 7, iter: 0, loss is: [0.29835087]
epoch: 8, iter: 0, loss is: [0.26178697]
epoch: 9, iter: 0, loss is: [0.27210417]
epoch: 10, iter: 0, loss is: [0.2922308]
epoch: 11, iter: 0, loss is: [0.24840203]
epoch: 12, iter: 0, loss is: [0.28776324]
epoch: 13, iter: 0, loss is: [0.2649998]
epoch: 14, iter: 0, loss is: [0.24687095]
epoch: 15, iter: 0, loss is: [0.2222822]
epoch: 16, iter: 0, loss is: [0.25335276]
epoch: 17, iter: 0, loss is: [0.24512134]
epoch: 18, iter: 0, loss is: [0.22216968]
epoch: 19, iter: 0, loss is: [0.21300644]
epoch: 20, iter: 0, loss is: [0.20396289]
epoch: 21, iter: 0, loss is: [0.21724041]
epoch: 22, iter: 0, loss is: [0.2019036]
epoch: 23, iter: 0, loss is: [0.18444832]
epoch: 

## 5 保存并测试模型
在完成两层循环的训练过程后，将模型当前的参数（model.state_dict()）保存到文件中（通过参数指定保存的文件名 LR_model），以备预测或校验的程序调用。

In [6]:
# 定义飞桨动态图工作环境
with fluid.dygraph.guard():
    # 保存模型参数，文件名为LR_model
    # paddle.fluid.dygraph.save_dygraph(state_dict, model_path)
    # state_dict 是通过 Layer 的 state_dict() 方法得到的。
    fluid.save_dygraph(model.state_dict(), 'LR_model')
    print("模型保存成功，模型参数保存在LR_model中")
    
#保存训练参数到指定路径中，构建一个专门用预测的program
fluid.io.save_inference_model("./boston_model1",   #保存推理model的路径
                                  ['x'],            #推理（inference）需要 feed 的数据
                                  [y_predict],      #保存推理（inference）结果的 Variables
                                  exe)              #exe 保存 inference model
fluid.io.save_params(exe,"./boston_model1")

模型保存成功，模型参数保存在LR_model中


NameError: name 'y_predict' is not defined

回顾下基于飞桨实现的房价预测模型，实现的效果与之前基于Python实现的模型没有区别，但两者的实现成本有天壤之别。飞桨的愿景是用户只需要了解模型的逻辑概念，不需要关心实现细节，就能搭建强大的模型。

下面我们选择一条数据样本，测试下模型的预测效果。 测试过程和在应用场景中使用模型的过程是一致的，可分成三个主要步骤。

首先，**配置模型预测的机器资源**，本案例默认使用本机，所以无需写代码指定。    
其次，**将训练好的模型参数加载到模型实例中**，由两个语句完成，第一句是从文件中读取模型参数，第二句是将参数内容加载到模型。加载完毕后，需要将模型的状态调整为“校验”（evalueation）。这是因为训练状态的模型需要同时支持前向计算和反向传导梯度，模型的实现较为臃肿。而校验/预测状态的模型只需要支持前向计算，模型的实现更加简单，性能更好。  
最后，**将待预测的样本特征输入到模型中，打印输出的预测结果**。比较“模型预测值”和“真实房价”可见，模型预测的效果与真实房价接近。
load_one_example函数实现了从数据集中抽一条样本作为测试样本。

In [ ]:
def load_one_example(data_dir):
    f = open(data_dir, 'r')
    datas = f.readlines()
    # 选择倒数第10条数据用于测试
    tmp = datas[-10]
    tmp = tmp.strip().split()
    one_data = [float(v) for v in tmp]

    # 对数据进行归一化处理
    for i in range(len(one_data)-1):
        one_data[i] = (one_data[i] - avg_values[i]) / (max_values[i] - min_values[i])

    data = np.reshape(np.array(one_data[:-1]), [1, -1]).astype(np.float32)
    label = one_data[-1]
    return data, label

In [ ]:
with dygraph.guard():
    # 参数为保存模型参数的文件地址
    # 该接口尝试从磁盘中加载参数或优化器的 dict
    model_dict, _ = fluid.load_dygraph('LR_model')
    model.load_dict(model_dict)
    # 在预测的模式下，DyGraph将只会执行前向的预测网络，而不会进行自动求导并执行反向网络
    # 当我们需要在训练的过程中进行预测时需要使用YourModel.eval()切换到预测模式，
    # 并且在预测完成后使用YourModel.train()切换回训练模式继续训练
    model.eval()

    # 参数为数据集的文件地址
    test_data, label = load_one_example('./housing.data')
    # 将数据转为动态图的variable格式
    # value (ndarray) 需要转换的numpy.ndarray对象
    test_data = dygraph.to_variable(test_data)
    results = model(test_data)

    # 对结果做反归一化处理
    results = results * (max_values[-1] - min_values[-1]) + avg_values[-1]
    print("Inference result is {}, the corresponding label is {}".format(results.numpy(), label))